In [ ]:

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Tuple, List, Dict
import random
import re
from textblob import TextBlob
from torch.quantization import quantize_dynamic
import time
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Device and model configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "./fine_tuned_personality_bot/"  # Update with your model path

class PersonalityBot:
    def __init__(self, model_path: str = MODEL_PATH):
        self.model_path = model_path
        self.model, self.tokenizer = self.setup_model()
        
    def setup_model(self) -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
        """Initialize and configure the model and tokenizer."""
        logger.info(f"Setting up model from {self.model_path}")
        
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found at {self.model_path}")
        
        try:
            tokenizer = AutoTokenizer.from_pretrained(
                self.model_path,
                use_fast=True
            )
            tokenizer.pad_token = tokenizer.eos_token
            
            model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                low_cpu_mem_usage=True,
                use_safetensors=True,
                device_map="auto"
            )
            
            if device.type == "cpu":
                model = quantize_dynamic(
                    model,
                    {torch.nn.Linear},
                    dtype=torch.qint8
                ).to(device)
            
            model.eval()
            logger.info("Model setup completed successfully")
            return model, tokenizer
            
        except Exception as e:
            logger.error(f"Model setup failed: {str(e)}")
            raise

    def categorize_prompt(self, prompt: str) -> str:
        """Categorize input prompt for contextual response generation."""
        categories: Dict[str, List[str]] = {
            "market_analysis": [
                "price", "market", "chart", "analysis", "trend", "prediction",
                "bull", "bear", "trading", "volume"
            ],
            "tech_discussion": [
                "blockchain", "protocol", "layer", "scaling", "code", "development",
                "smart contract", "gas", "network"
            ],
            "defi": [
                "defi", "yield", "farming", "liquidity", "stake", "lending",
                "borrow", "apy", "tvl"
            ],
            "nft": [
                "nft", "art", "collectible", "mint", "opensea", "rarity",
                "floor price", "pfp"
            ],
            "culture": [
                "community", "dao", "governance", "vote", "proposal",
                "alpha", "degen", "fud", "fomo"
            ]
        }
        
        prompt_lower = prompt.lower()
        for category, keywords in categories.items():
            if any(keyword in prompt_lower for keyword in keywords):
                return category
        return "general"

    def generate_hook(self, category: str) -> str:
        """Generate category-specific attention hooks."""
        hooks = {
            "market_analysis": [
                "Market alert:", "Chart check:", "Price watch:",
                "Trading insight:", "Market alpha:"
            ],
            "tech_discussion": [
                "Tech deep dive:", "Builder's corner:", "Protocol watch:",
                "Dev update:", "Architecture take:"
            ],
            "defi": [
                "DeFi alpha:", "Yield watch:", "Smart money move:",
                "Protocol alert:", "TVL update:"
            ],
            "nft": [
                "NFT alpha:", "Collection watch:", "Mint alert:",
                "Floor check:", "Digital art take:"
            ],
            "culture": [
                "Culture take:", "DAO watch:", "Governance alert:",
                "Community vibe:", "Alpha leak:"
            ],
            "general": [
                "Hot take:", "Unpopular opinion:", "Plot twist:",
                "Real talk:", "Quick thought:"
            ]
        }
        
        category_hooks = hooks.get(category, hooks["general"])
        return random.choice(category_hooks) if random.random() < 0.4 else ""

    def add_emojis(self, text: str, category: str) -> str:
        """Add contextual emojis based on content category."""
        emoji_sets = {
            "market_analysis": ["📈", "📊", "💹", "📉", "💸", "🎯", "📱"],
            "tech_discussion": ["⚡️", "🔧", "💻", "🛠️", "🔨", "🧮", "🔋"],
            "defi": ["🏦", "💰", "🏧", "💳", "🔄", "⚖️", "🎰"],
            "nft": ["🎨", "🖼️", "🎭", "🎪", "🎟️", "🎮", "🃏"],
            "culture": ["🌐", "🤝", "🗣️", "🎭", "🎪", "🎯", "🎲"],
            "general": ["🚀", "💎", "🌙", "🔥", "💡", "🎯", "⭐️"]
        }
        
        category_emojis = emoji_sets.get(category, emoji_sets["general"])
        emoji_count = random.randint(1, 2)
        chosen_emojis = random.sample(category_emojis, emoji_count)
        
        return f"{text} {''.join(chosen_emojis)}"

    def generate_engagement_phrase(self, category: str) -> str:
        """Generate contextual engagement prompts."""
        phrases = {
            "market_analysis": [
                "What's your price target?",
                "Bulls or bears?",
                "Who's buying this dip?",
                "Thoughts on this setup?"
            ],
            "tech_discussion": [
                "Devs, thoughts?",
                "Valid architecture?",
                "Spotted any issues?",
                "Who's building similar?"
            ],
            "defi": [
                "What's your yield strategy?",
                "Aping in?",
                "Found better rates?",
                "Risk level?"
            ],
            "nft": [
                "Cope or hope?",
                "Floor predictions?",
                "Mining this?",
                "Art or utility?"
            ],
            "culture": [
                "Based or nah?",
                "Who else sees this?",
                "Your governance take?",
                "DAO voters wya?"
            ],
            "general": [
                "Thoughts?",
                "Based?",
                "Who's with me?",
                "Change my mind."
            ]
        }
        
        category_phrases = phrases.get(category, phrases["general"])
        return random.choice(category_phrases) if random.random() < 0.3 else ""

    def add_hashtags(self, text: str, category: str) -> str:
        """Add relevant hashtags based on content and character limit."""
        hashtags = {
            "market_analysis": [
                "#CryptoTrading", "#TechnicalAnalysis", "#CryptoMarkets",
                "#Trading", "#Charts", "#PriceAction"
            ],
            "tech_discussion": [
                "#Blockchain", "#CryptoTech", "#Web3Dev", "#DLT",
                "#SmartContracts", "#BuilderSpace"
            ],
            "defi": [
                "#DeFi", "#YieldFarming", "#Staking", "#DeFiSeason",
                "#PassiveIncome", "#DeFiYield"
            ],
            "nft": [
                "#NFTs", "#NFTCommunity", "#NFTCollector", "#NFTArt",
                "#NFTProject", "#TokenizedArt"
            ],
            "culture": [
                "#CryptoCulture", "#DAOs", "#Web3", "#CryptoTwitter",
                "#CryptoLife", "#BuildingWeb3"
            ],
            "general": [
                "#Crypto", "#Web3", "#Bitcoin", "#Ethereum",
                "#CryptoTwitter", "#BuildingTheFuture"
            ]
        }
        
        remaining_chars = 280 - len(text)
        if remaining_chars < 15:  # Not enough space for hashtags
            return text
            
        category_hashtags = hashtags.get(category, hashtags["general"])
        selected_hashtags = []
        
        # Add 1-2 hashtags while respecting character limit
        for _ in range(random.randint(1, 2)):
            if not category_hashtags or remaining_chars < 15:
                break
            hashtag = random.choice(category_hashtags)
            if len(hashtag) + 1 <= remaining_chars:
                selected_hashtags.append(hashtag)
                category_hashtags.remove(hashtag)
                remaining_chars -= len(hashtag) + 1
        
        return f"{text} {' '.join(selected_hashtags)}"

    def clean_response(self, text: str, category: str) -> str:
        """Clean and format the response for Twitter."""
        # Remove unwanted patterns
        text = re.sub(r'@\w+|http\S+|\[.*?\]|\(.*?\)|"|Q:|A:', '', text)
        text = re.sub(r'\b(Note|Example|Rules).*?(?=[.!?]|$)', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Split into sentences and filter
        sentences = re.split(r'(?<=[.!?])\s+', text)
        sentences = [s for s in sentences if len(s) > 20 and re.search(r'[.!?]$', s)]
        
        if not sentences:
            return self.get_fallback(category)
        
        # Combine sentences with proper spacing
        response = ' '.join(sentences[:2])
        
        # Add engaging elements
        hook = self.generate_hook(category)
        if hook:
            response = f"{hook} {response}"
        
        engagement = self.generate_engagement_phrase(category)
        if engagement:
            response = f"{response} {engagement}"
        
        return response[:240]  # Leave room for hashtags/emojis

    def get_fallback(self, category: str) -> str:
        """Generate category-specific fallback responses."""
        fallbacks = {
            "market_analysis": [
                "Charts looking juicy today! Anyone else seeing this setup? 📈",
                "Market's giving mixed signals but the volume tells a different story 👀"
            ],
            "tech_discussion": [
                "Sometimes the best protocols are the ones no one's talking about yet 🛠️",
                "Imagine still building without considering Layer 2 scaling 💻"
            ],
            "defi": [
                "Your yields are only as good as your risk management 🏦",
                "DeFi summer never ended, we just got better at farming 🌾"
            ],
            "nft": [
                "Art is subjective, but floor prices aren't 🎨",
                "Your NFT portfolio tells a story. Make it a good one 🖼️"
            ],
            "culture": [
                "Web3 culture is what we make it. Build accordingly 🌐",
                "Sometimes the real alpha is the friends we made along the way 🤝"
            ],
            "general": [
                "Just caught myself thinking about the future of crypto while making coffee ☕️",
                "Your portfolio is only as strong as your conviction 💎"
            ]
        }
        
        category_fallbacks = fallbacks.get(category, fallbacks["general"])
        return random.choice(category_fallbacks)

    def filter_tone(self, response: str) -> str:
        """Filter response tone and adjust if needed."""
        sentiment = TextBlob(response).sentiment
        
        if sentiment.polarity < -0.3:
            return self.get_fallback("general")
        
        if sentiment.subjectivity > 0.8:
            # Too subjective, add a disclaimer
            return f"Not financial advice but... {response}"
            
        return response

    def generate_response(self, prompt: str) -> str:
        """Generate a complete Twitter-ready response."""
        try:
            category = self.categorize_prompt(prompt)
            
            instruction = (
                
            
                "You are a witty crypto influencer known for sharp insights and engaging tweets. "
                "Your style combines tech expertise with modern internet culture. "
                f"You're currently discussing {category}-related topics. "
                "Write in a confident, casual voice using current crypto/tech slang. "
                "Keep responses punchy and memorable - perfect for viral tweets. "
                "Focus on providing unique perspectives that spark discussion. "
                "Mix humor with genuine insight, avoiding generic takes. "
                "Use conversational tone but maintain authority on the subject."
            )
            
            context = f"{instruction}\n\nQ: {prompt}\nA:"
            
            inputs = self.tokenizer(
                context,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=256
            ).to(device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=100,
                    do_sample=True,
                    temperature=0.92,
                    top_k=50,
                    top_p=0.95,
                    repetition_penalty=1.2,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )
            
            response = self.tokenizer.decode(
                outputs[0],
                skip_special_tokens=True
            ).split("A:")[-1].strip()
            
            if not response or len(response) < 20:
                return self.get_fallback(category)
            
            # Apply enhancements
            response = self.clean_response(response, category)
            response = self.filter_tone(response)
            response = self.add_emojis(response, category)
            response = self.add_hashtags(response, category)
            
            return response
            
        except Exception as e:
            logger.error(f"Error generating response: {str(e)}")
            return self.get_fallback("general")

def main():
    """Main execution function."""
    try:
        bot = PersonalityBot()
        logger.info("Bot initialized successfully")
        
        #Test prompts
        
        test_prompts = [
            "How do gas fees compare between Ethereum and Solana?",
            "Tell me why NFTs are either a scam or the future.",
            "What's your take on Bitcoin as digital gold?",
            "Explain staking in the context of DeFi but make it funny.",
            "Give me a wild prediction for crypto in 2030.",
            "What's a DAO, and should I care?",
            "Can you explain why everyone's obsessed with Layer 2 solutions?",
            "What's the wildest crypto meme you've seen this week?",
            "How do I avoid FOMO during bull runs?",
            "Is it too late to get into crypto? Be honest!",
            "How would you describe Web3 to someone who barely uses the internet?",
            "What's your hottest take on decentralized finance?"
        ]
        
        # Test run with basic prompts
        logger.info("Starting test run with sample prompts...")
        for i, prompt in enumerate(test_prompts, 1):
            # logger.info(f"\nProcessing test prompt {i}/{len(test_prompts)}")
            print(f"\nPrompt {i}: {prompt}")
            
            start_time = time.time()
            response = bot.generate_response(prompt)
            elapsed_time = time.time() - start_time
            
            print(f"Response: {response}")
            print("")
            print(f"Runtime: {elapsed_time:.2f} seconds")
            
       
        
        # Interactive mode/twitter integrate
        logger.info("Entering interactive mode...")
        print("\nEnter your prompts (type 'quit' to exit):")
        
        while True:
            user_prompt = input("\nYour prompt: ").strip()
            
            if user_prompt.lower() == 'quit':
                print("Exiting... Thanks for using CryptoPersonalityBot!")
                break
                
            if not user_prompt:
                print("Please enter a valid prompt!")
                continue
            
            try:
                start_time = time.time()
                response = bot.generate_response(user_prompt)
                elapsed_time = time.time() - start_time
                
                print(f"Response: {response}")
                print(f"Runtime: {elapsed_time:.2f} seconds")
                
            except Exception as e:
                logger.error(f"Error processing prompt: {str(e)}")
                print("Oops! Something went wrong. Please try again.")
                
    except Exception as e:
        logger.error(f"Application error: {str(e)}")
        print("Critical error occurred. Please check the logs for details.")
        
if __name__ == "__main__":
    main()

2024-11-19 09:08:36,895 - INFO - Setting up model from ./fine_tuned_personality_bot/
2024-11-19 09:08:38,132 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2024-11-19 09:08:59,109 - WARNING - Some parameters are on the meta device because they were offloaded to the cpu.
2024-11-19 09:08:59,111 - INFO - Model setup completed successfully
2024-11-19 09:08:59,111 - INFO - Bot initialized successfully
2024-11-19 09:08:59,112 - INFO - Starting test run with sample prompts...



Prompt 1: How do gas fees compare between Ethereum and Solana?
Response: Y'all want to know about gas fees? It's like asking which pizza topping is better – ETH or SOL! 💻🔨 #CryptoTech

Runtime: 30.20 seconds

Prompt 2: Tell me why NFTs are either a scam or the future.
Response: Like, they're already changing game, making old art & collectibles hot again. But let's be real – most ppl minting 'em don't even understand what they're doing. 🎨 #NFTArt #NFTProject

Runtime: 26.15 seconds

Prompt 3: What's your take on Bitcoin as digital gold?
Response: Not financial advice but... Your portfolio is only as strong as your conviction 💎 🌙 #Web3 #Ethereum

Runtime: 28.17 seconds

Prompt 4: Explain staking in the context of DeFi but make it funny.
Response: Your yields are only as good as your risk management 🏦 🎰 #YieldFarming #DeFiYield

Runtime: 28.84 seconds

Prompt 5: Give me a wild prediction for crypto in 2030.
Response: Lowkey thinking we'll see MFTs replace ETH as devs' go-to choice by '30